In [50]:
import time
import pandas as pd
from langchain_ollama import ChatOllama
from tqdm import tqdm
import subprocess
import psutil

In [51]:
questions_df = pd.read_csv("questions.csv")
questions = questions_df["questions"].tolist()

In [52]:
models = ["qwen2.5:72b"]

In [53]:
def get_gpu_memory_usage():
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.used,memory.total', '--format=csv,noheader,nounits'],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    # Чтение результата
    gpu_usage = result.stdout.strip().split('\n')
    used, total = map(int, gpu_usage[0].split(', '))
    return used, total

In [54]:
def test_model(model, questions, model_name):
    total_time = 0
    memory_usage = []

    # Создаем DataFrame для записи результатов
    columns = [
        "question", "answer", "time_taken", 
        "gpu_memory_before", "gpu_memory_after", "gpu_memory_delta"
    ]
    model_df = pd.DataFrame(columns=columns)

    # Инициализируем файл для результатов
    model_df.to_csv(f"{model_name}_results.csv", mode='w', header=True, index=False)

    # Начинаем тестирование
    for question in tqdm(questions):
        # Засекаем время
        start_time = time.time()

        # Получаем использование видеопамяти до запроса
        gpu_used_before, gpu_total = get_gpu_memory_usage()

        # Запрос к модели
        print(question)
        response = model.invoke(question)
        print(response)

        # Засекаем время окончания
        end_time = time.time()
        elapsed_time = end_time - start_time
        total_time += elapsed_time

        # Получаем использование видеопамяти после запроса
        gpu_used_after, _ = get_gpu_memory_usage()

        # Создаем строку результата для текущего вопроса
        result = {
            "question": question,
            "answer": response,
            "time_taken": elapsed_time,
            "gpu_memory_before": gpu_used_before,
            "gpu_memory_after": gpu_used_after,
            "gpu_memory_delta": gpu_used_after - gpu_used_before
        }

        # Добавляем строку в DataFrame
        model_df = model_df.append(result, ignore_index=True)

        # Записываем данные в CSV файл сразу после добавления ответа
        model_df.to_csv(f"{model_name}_results.csv", mode='a', header=False, index=False)

        # Добавляем информацию о потреблении ресурсов
        memory_usage.append({
            "time": elapsed_time,
            "gpu_memory_before": gpu_used_before,
            "gpu_memory_after": gpu_used_after,
            "gpu_memory_delta": gpu_used_after - gpu_used_before,
            "cpu_usage": psutil.cpu_percent(interval=1),  # Загрузка CPU
            "ram_usage": psutil.virtual_memory().percent  # Использование RAM
        })

    # Сохраняем данные по использованию ресурсов
    resource_df = pd.DataFrame(memory_usage)
    resource_df.to_csv(f"{model_name}_resources.csv", index=False)

    return total_time

In [ ]:
# Словарь для хранения общего времени
total_time_all_models = {}

# Словарь для записи общего времени в CSV
overall_times = []

# Тестируем все модели
for model in models:
    print(f"Тестирование модели: {model}")
    llm_model = ChatOllama(model=model)
    
    total_time = test_model(llm_model, questions, model)
    total_time_all_models[model] = total_time
    overall_times.append({
        "model": model,
        "total_time": total_time
    })
    print(f"Общее время для модели {model}: {total_time} секунд")

# Выводим общее время на все вопросы
total_time_overall = sum(total_time_all_models.values())
print(f"Общее время на все вопросы для всех моделей: {total_time_overall} секунд")

# Сохраняем общее время в файл
overall_times_df = pd.DataFrame(overall_times)
overall_times_df.to_csv("overall_times.csv", index=False)

Тестирование модели: qwen2.5:72b


  0%|                                                    | 0/51 [00:00<?, ?it/s]

Что такое искусственный интеллект?
